In [30]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [31]:
from sklearn.metrics import mean_squared_log_error

In [32]:
train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv', parse_dates=['date'])
test = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv')
sample_sub = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv')

train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])

train['day_of_week'] = train['date'].dt.dayofweek
test['day_of_week'] = test['date'].dt.dayofweek

In [33]:
cutoff_date = train['date'].max() - pd.Timedelta(days=14)
train_part = train[train['date'] <= cutoff_date]
valid_part = train[train['date'] > cutoff_date]

In [34]:
median_sales = (
    train_part.groupby(['store_nbr', 'family', 'onpromotion', 'day_of_week'])['sales']
    .median()
    .reset_index()
    .rename(columns={'sales': 'median_sales'})
)

valid_merged = valid_part.merge(
    median_sales,
    on=['store_nbr', 'family', 'onpromotion', 'day_of_week'],
    how='left'
)

In [35]:
fallback1 = train_part.groupby(['store_nbr', 'family', 'day_of_week'])['sales'].median().reset_index()
fallback1 = fallback1.rename(columns={'sales':'median_sales_fallback'})
valid_merged = valid_merged.merge(
    fallback1, on=['store_nbr', 'family','day_of_week'], how='left'
)
valid_merged['median_sales'] = valid_merged['median_sales'].fillna(valid_merged['median_sales_fallback'])

global_median = train_part['sales'].median()
valid_merged['median_sales'] = valid_merged['median_sales'].fillna(global_median)


In [36]:
valid_preds = valid_merged['median_sales'].clip(0)
rmsle = np.sqrt(mean_squared_log_error(valid_merged['sales'], valid_preds))
print(f"Local Validation RMSLE: {rmsle:.4f}")


Local Validation RMSLE: 1.0267


In [37]:
train['day_of_week'] = train['date'].dt.dayofweek
test['day_of_week'] = test['date'].dt.dayofweek

median_sales_full = (
    train.groupby(['store_nbr', 'family', 'onpromotion', 'day_of_week'])['sales']
    .median()
    .reset_index()
    .rename(columns={'sales': 'median_sales'})
)

test_merged = test.merge(
    median_sales_full,
    on=['store_nbr', 'family', 'onpromotion', 'day_of_week'],
    how='left'
)

fallback_full = train.groupby(['store_nbr', 'family', 'day_of_week'])['sales'].median().reset_index()
fallback_full = fallback_full.rename(columns={'sales':'median_sales_fallback'})
test_merged = test_merged.merge(fallback_full, on=['store_nbr','family','day_of_week'], how='left')
test_merged['median_sales'] = test_merged['median_sales'].fillna(test_merged['median_sales_fallback'])
test_merged['median_sales'] = test_merged['median_sales'].fillna(train['sales'].median())


In [38]:
submission = sample_sub.copy()
submission['sales'] = test_merged['median_sales'].clip(0)
submission.to_csv("/kaggle/working/submission.csv", index=False)

print("submission.csv saved!")

submission.csv saved!


In [39]:
submission.head()

,id,sales
0,3000888,3.0
1,3000889,0.0
2,3000890,6.0
3,3000891,2631.0
4,3000892,0.0
